In [ ]:
import os
import pandas as pd
import re

def extract_camera_timestamps(root_folder):
    for scene_folder in os.listdir(root_folder):
        scene_path = os.path.join(root_folder, scene_folder)
        if os.path.isdir(scene_path):
            camera_df = pd.DataFrame(columns=['camera_timestamp'])

            camera_folder_name = None
            for dir_name in os.listdir(scene_path):
                if 'folder1' in dir_name:
                    camera_folder_name = os.path.join(scene_path, dir_name)
                    break

            if camera_folder_name:
                for file in os.listdir(camera_folder_name):
                    if file.lower().endswith(('.jpg', '.jpeg')):
                        # Extract the camera timestamp from the file name using regular expressions
                        timestamp_match = re.search(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}\.\d{6}', file)
                        if timestamp_match:
                            camera_timestamp = timestamp_match.group(0)
                            # Remove the date part from the timestamp and replace '-' with ':'
                            camera_timestamp = camera_timestamp.replace('2023-08-26-', '').replace('-', ':')
                            camera_df = camera_df.append({'camera_timestamp': camera_timestamp}, ignore_index=True)

            # Save camera timestamps CSV in the scene folder
            camera_timestamps_file = os.path.join(scene_path, 'camera_timestamps.csv')
            camera_df.to_csv(camera_timestamps_file, index=False)

def extract_radar_timestamps(root_folder):
    for scene_folder in os.listdir(root_folder):
        scene_path = os.path.join(root_folder, scene_folder)
        if os.path.isdir(scene_path):
            radar_df = pd.DataFrame(columns=['radar_timestamp'])

            radar_folder_name = None
            for dir_name in os.listdir(scene_path):
                if '20' in dir_name:
                    radar_folder_name = os.path.join(scene_path, dir_name)
                    break

            if radar_folder_name:
                for file in os.listdir(radar_folder_name):
                    if file.lower().endswith('.csv'):
                        # Extract the radar timestamp from the file name and replace '-' with ':'
                        radar_timestamp = os.path.splitext(file)[0].replace('-', ':')
                        radar_df = radar_df.append({'radar_timestamp': radar_timestamp}, ignore_index=True)

            # Save radar timestamps CSV in the scene folder
            radar_timestamps_file = os.path.join(scene_path, 'radar_timestamps.csv')
            radar_df.to_csv(radar_timestamps_file, index=False)

def synchronize_timestamps(camera_root_folder, radar_root_folder):
    for scene_folder in os.listdir(radar_root_folder):
        scene_path = os.path.join(radar_root_folder, scene_folder)
        if os.path.isdir(scene_path):
            camera_timestamps_file = os.path.join(camera_root_folder, scene_folder, 'camera_timestamps.csv')
            radar_timestamps_file = os.path.join(scene_path, 'radar_timestamps.csv')

            if os.path.exists(camera_timestamps_file) and os.path.exists(radar_timestamps_file):
                # Read camera timestamps from the scene folder
                camera_df = pd.read_csv(camera_timestamps_file)
                radar_df = pd.read_csv(radar_timestamps_file)

                camera_df['camera_timestamp'] = pd.to_datetime(camera_df['camera_timestamp'], format='%H:%M:%S.%f')
                radar_df['radar_timestamp'] = pd.to_datetime(radar_df['radar_timestamp'], format='%H:%M:%S.%f')

                synchronized_df = pd.DataFrame(columns=['camera_timestamp', 'closest_radar_timestamp'])

                for camera_ts in camera_df['camera_timestamp']:
                    closest_radar_idx = (radar_df['radar_timestamp'] - camera_ts).abs().idxmin()
                    closest_radar_ts = radar_df.loc[closest_radar_idx, 'radar_timestamp']
                    synchronized_df = synchronized_df.append({'camera_timestamp': camera_ts, 'closest_radar_timestamp': closest_radar_ts}, ignore_index=True)

                synchronized_df['camera_timestamp'] = synchronized_df['camera_timestamp'].dt.strftime('%H:%M:%S.%f')
                synchronized_df['closest_radar_timestamp'] = pd.to_datetime(synchronized_df['closest_radar_timestamp']).dt.strftime('%H:%M:%S.%f')

                # Use the scene folder name in the output CSV file name
                synchronized_timestamps_file = os.path.join(scene_path, f'Radar_Camera_Synch_{scene_folder}.csv')
                synchronized_df.to_csv(synchronized_timestamps_file, index=False)

# Specify the root folder for the camera (including sub-folders)
camera_root_folder = '/media/nitish/One Touch/extracted'

# Specify the root folder for the radar
radar_root_folder = '26th_aug'

# Call the functions to extract and synchronize timestamps
extract_camera_timestamps(camera_root_folder)
extract_radar_timestamps(radar_root_folder)
synchronize_timestamps(camera_root_folder, radar_root_folder)